In [1]:
import os
from placesApi import PlacesApi
from utils import distance, create_places_table, select_all_places, insert_place

## test Google PLaces API

In [13]:
# Access the environment variable
ABIR_API_KEY = os.environ.get("ABIR_API_KEY")
API_KEY = os.environ.get("GOOGLE_API_KEY")
api = PlacesApi(ABIR_API_KEY)

In [3]:
# Location coordinates and radius
latitude = 48.752794
longitude = 2.398159
radius = 2000.0  # meters, max=50000 (50km)
# Request parameters
maxResults = 20 # max=20
rankBy = "POPULARITY" # "POPULARITY" or "DISTANCE"
#rankPreference": ""
includedTypes = ["train_station","subway_station","bus_station","bus_stop" ] #"hospital","shopping_mall", "school"
fieldMask = ["places.primaryType",
             "places.displayName",
             'places.types',
             "places.location"]
includedPrimaryTypes = None

In [4]:
response = api.searchNearby(latitude=latitude, longitude=longitude, radius=radius,
                            fieldMask= fieldMask, includedTypes= includedTypes,
                            includedPrimaryTypes = includedPrimaryTypes, maxResults=20, rankPreference=rankBy) 

In [5]:
places = response.json()['places']
print("Places found: ", len(places))
print("First place:")
places[0]


Places found:  20
First place:


{'types': ['transit_station',
  'bus_stop',
  'train_station',
  'point_of_interest',
  'establishment'],
 'location': {'latitude': 48.764503000000005, 'longitude': 2.4108400999999997},
 'displayName': {'text': 'Choisy-Le-Roi', 'languageCode': 'fr'},
 'primaryType': 'transit_station'}

## Call API on dataset and save into SQL DB

In [6]:
create_places_table()

PostgreSQL connection is closed


In [7]:
import pandas as pd
df = pd.read_csv('../data/ads_clean.csv')

In [14]:
for i,row in df.iloc[2000:7523].iterrows():
    
    #row['latitude'],row['longitude']
    # Get nearby stations
    response = api.searchNearby(latitude=row['latitude'], longitude=row['longitude'], radius=radius,
                            fieldMask= fieldMask, includedTypes= includedTypes,
                            includedPrimaryTypes = includedPrimaryTypes, maxResults=20, rankPreference=rankBy) 
    
    if not response.json():
        continue
    
    for place in response.json()['places']:       
        place_to_add = {
            'id':row['id'],
            'primaryType': place['primaryType'],
            'latitude': place['location']['latitude'],
            'longitude': place['location']['longitude'],
            'distance': distance(row['latitude'], row['longitude'], place['location']['latitude'], place['location']['longitude']),
            'name': place['displayName']['text'],
            'types': ';'.join(place['types']),       
            }
        place_to_add['id'] = row['id']
        place_to_add['distance'] = distance(row['latitude'], row['longitude'], place['location']['latitude'], place['location']['longitude'])
        insert_place(place_to_add)
    
    print("finished row: ", i)

finished row:  2000
finished row:  2001
finished row:  2002
finished row:  2003
finished row:  2004
finished row:  2005
finished row:  2006
finished row:  2007
finished row:  2008
finished row:  2009
finished row:  2010
finished row:  2011
finished row:  2012
finished row:  2013
finished row:  2014
finished row:  2015
finished row:  2016
finished row:  2017
finished row:  2018
finished row:  2019
finished row:  2020
finished row:  2021
finished row:  2022
finished row:  2023
finished row:  2024
finished row:  2025
finished row:  2026
finished row:  2027
finished row:  2028
finished row:  2029
finished row:  2030
finished row:  2031
finished row:  2032
finished row:  2033
finished row:  2034
finished row:  2035
finished row:  2036
finished row:  2037
finished row:  2038
finished row:  2039
finished row:  2040
finished row:  2041
finished row:  2042
finished row:  2043
finished row:  2044
finished row:  2045
finished row:  2046
finished row:  2047
finished row:  2048
finished row:  2049


In [15]:
_ = select_all_places(save=True)

PostgreSQL connection is closed
